**Homework_1**

In [2]:
import nltk
import ssl

from corus import load_lenta


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from pymorphy2 import MorphAnalyzer

from tqdm import tqdm

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [3]:
#загрузим датасет

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [4]:
# сформируем датасет из текста новостей

dataset = [next(records).text for i in range(3000)]
dataset[0][:1000]

'Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Хохфильцене вечером 12 декабря. Как написал биатлонист Александр Логинов, их считают виновными в махинациях с переливанием крови. Биатлонисту Антону Шипулину, также попавшему в список, полиция нанесла отдельный визит: сейчас он тренируется отдельно в австрийском Обертиллахе. Обвинения спортсмен назвал бредом, а также указал на «охоту на ведьм» в мировом биатлоне. В Австрии прием допинга — уголовное пр

**Предобработка датасета**
1. Нормализация

In [5]:
normalized_texts = [text.lower() for text in dataset]

2. Токенизация с учетом языка слова (английский или русский) и удаление стоп-слов

In [ ]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')

In [7]:
#списки стоп-слов для русского и английского
russian_stop_words = set(stopwords.words('russian'))
english_stop_words = set(stopwords.words('english'))

In [8]:
#функция для определения языка слова

def detect_language(word):

    if all('а' <= char <= 'я' for char in word):
        return 'russian'

    if all('a' <= char <= 'z' for char in word):
        return 'english'

In [9]:
tokenized_texts = []

for text in normalized_texts:

    tokens = word_tokenize(text)
    filtered_tokens = []

    for token in tokens:

        language = detect_language(token)

        if language == 'russian':

            if token not in russian_stop_words:

                filtered_tokens.append(token)

        elif language == 'english':

            if token not in english_stop_words:

                filtered_tokens.append(token)

    tokenized_texts.append(filtered_tokens)

In [10]:
tokenized_texts[0][:30]

['австрийские',
 'правоохранительные',
 'органы',
 'представили',
 'доказательств',
 'нарушения',
 'российскими',
 'биатлонистами',
 'антидопинговых',
 'правил',
 'сообщил',
 'посол',
 'россии',
 'вене',
 'дмитрий',
 'любинский',
 'итогам',
 'встречи',
 'уполномоченного',
 'адвоката',
 'дипмиссии',
 'представителями',
 'прокуратуры',
 'страны',
 'передает',
 'тасс',
 'действует',
 'презумпция',
 'невиновности',
 'ограничений']

3. Лемматизация

In [11]:
morph = MorphAnalyzer()

lemmatized_texts = []

for tokens in tqdm(tokenized_texts):
    lemmas = [morph.parse(word)[0].normal_form for word in tokens]
    lemmatized_texts.append(lemmas)

100%|██████████| 3000/3000 [00:29<00:00, 100.44it/s]


In [12]:
lemmatized_texts[0][:30]

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило',
 'сообщить',
 'посол',
 'россия',
 'вена',
 'дмитрий',
 'любинский',
 'итог',
 'встреча',
 'уполномоченный',
 'адвокат',
 'дипмиссия',
 'представитель',
 'прокуратура',
 'страна',
 'передавать',
 'тасс',
 'действовать',
 'презумпция',
 'невиновность',
 'ограничение']

4. Стемминг

In [13]:
stemmer = SnowballStemmer("russian")

stemmed_texts = []


for tokens in tokenized_texts:
    stems = [stemmer.stem(word) for word in tokens]
    stemmed_texts.append(stems)


In [14]:
stemmed_texts[0][:30]

['австрийск',
 'правоохранительн',
 'орга',
 'представ',
 'доказательств',
 'нарушен',
 'российск',
 'биатлонист',
 'антидопингов',
 'прав',
 'сообщ',
 'посол',
 'росс',
 'вен',
 'дмитр',
 'любинск',
 'итог',
 'встреч',
 'уполномочен',
 'адвокат',
 'дипмисс',
 'представител',
 'прокуратур',
 'стран',
 'переда',
 'тасс',
 'действ',
 'презумпц',
 'невиновн',
 'ограничен']